In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

In [2]:
from scispacy.data_util import read_full_med_mentions, read_ner_from_tsv
from scispacy.per_class_scorer import PerClassScorer
from scispacy.train_utils import evaluate_ner

In [ ]:
def train_ner(output_dir: str,
              train_data_path: str,
              dev_data_path: str,
              test_data_path: str,
              run_test: bool = None,
              model: str = None,
              n_iter: int = 10,
              meta_overrides: str = None):

    util.fix_random_seed(util.env_opt("seed", 0))
    train_data = read_ner_from_tsv(train_data_path)
    dev_data = read_ner_from_tsv(dev_data_path)
    test_data = read_ner_from_tsv(test_data_path)
    os.makedirs(output_dir, exist_ok=True)
    if run_test:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
        evaluate_ner(nlp, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
        evaluate_ner(nlp, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))
    else:
        train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides)

In [ ]:
def train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides):
    """Load the model, set up the pipeline and train the entity recognizer."""
    
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    if meta_overrides is not None:
        metadata = json.load(open(meta_overrides))
        nlp.meta.update(metadata)

    original_tokenizer = nlp.tokenizer

#     nlp.tokenizer = nlp_en.tokenizer

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names and "parser" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="parser")
    elif 'ner' not in nlp.pipe_names and "tagger" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="tagger")
    elif 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    dropout_rates = util.decaying(util.env_opt('dropout_from', 0.2),
                                  util.env_opt('dropout_to', 0.2),
                                  util.env_opt('dropout_decay', 0.005))
    batch_sizes = util.compounding(util.env_opt('batch_from', 1),
                                   util.env_opt('batch_to', 32),
                                   util.env_opt('batch_compound', 1.001))

    optimizer = nlp.begin_training()
    best_epoch = 0
    best_f1 = 0
    for i in range(n_iter):
        print(str(i)+'--'+str(n_iter))
        random.shuffle(train_data)
        count = 0
        losses = {}
        total = len(train_data)

        with nlp.disable_pipes(*other_pipes):  # only train NER
            with tqdm.tqdm(total=total, leave=True) as pbar:
                for batch in minibatch(train_data, size=batch_sizes):
                    docs, golds = zip(*batch)
                    nlp.update(docs, golds, sgd=optimizer,
                               losses=losses, drop=next(dropout_rates))
                    pbar.update(len(batch))
                    if count % 100 == 0 and count > 0:
                        print('sum loss: %s' % losses['ner'])
                    count += 1

        # save model to output directory
        output_dir_path = Path(output_dir + "/" + str(i))
        if not output_dir_path.exists():
            output_dir_path.mkdir()

        with nlp.use_params(optimizer.averages):
            nlp.tokenizer = original_tokenizer
            nlp.to_disk(output_dir_path)
            print("Saved model to", output_dir_path)

        # test the saved model
        print("Loading from", output_dir_path)
        nlp2 = util.load_model_from_path(output_dir_path)
#         nlp2.tokenizer = nlp_en.tokenizer

        metrics = evaluate_ner(nlp2, dev_data)
        if metrics["f1-measure-overall"] > best_f1:
            best_f1 = metrics["f1-measure-overall"]
            best_epoch = i
    # save model to output directory
    best_model_path = Path(output_dir + "/" + "best")
    print(f"Best Epoch: {best_epoch} of {n_iter}")
    if os.path.exists(best_model_path):
        shutil.rmtree(best_model_path)
    shutil.copytree(os.path.join(output_dir, str(best_epoch)),
                    best_model_path)

    # test the saved model
    print("Loading from", best_model_path)
    nlp2 = util.load_model_from_path(best_model_path)
#     nlp2.tokenizer = nlp_en.tokenizer

    evaluate_ner(nlp2, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
    evaluate_ner(nlp2, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))

In [ ]:
##### Genes ####
model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-BC2GM-en-pubmed-pmc-lg/'
train_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/BC2GM-IOB/train.tsv'
dev_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/BC2GM-IOB/devel.tsv'
test_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/BC2GM-IOB/test.tsv'
run_test = False
model_path = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc/' # None #'en_core_sci_md'
iterations = 7
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'

In [ ]:
train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)

In [ ]:
##### Diseases ####
  
model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-NCBI-en-pubmed-pmc-lg/'
train_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/NCBI-disease-IOB/train.tsv'
dev_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/NCBI-disease-IOB/devel.tsv'
test_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/NCBI-disease-IOB/test.tsv'
run_test = False
model_path = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc/' # None #'en_core_sci_md'
iterations = 7
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'


train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)



In [ ]:
##### Organisms ####

model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-linnaeus-en-pubmed-pmc-lg/'
train_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/linnaeus-IOB/train.tsv'
dev_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/linnaeus-IOB/devel.tsv'
test_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/linnaeus-IOB/test.tsv'
run_test = False
model_path = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc/' # None #'en_core_sci_md'
iterations = 7
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'


train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)


In [3]:
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize


def convert2IOB(text_data, ner_tags):
    tokenizer = WordPunctTokenizer()

    tokens = []
    ners = []
    spans = []

    split_text = tokenizer.tokenize(text_data)
    span_text = list(tokenizer.span_tokenize(text_data))
    # for each word token append 'O'
    arr = ['O'] * len(split_text)

    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
        except:
            pass

    elif isinstance(ner_tags, float) or ner_tags is None:
        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

    for each_tag in ner_tags:
        span_list = (each_tag[0], each_tag[1])
        token_list = wordpunct_tokenize(each_tag[2])
        ner_list = wordpunct_tokenize(each_tag[3])

        if (len(token_list) > len(ner_list)):
            ner_list = len(token_list) * ner_list
        for i in range(0, len(ner_list)):
            # The logic here is look for the first B-tag and then append I-tag next
            if (i == 0):
                ner_list[i] = 'B-' + ner_list[i]
            else:
                ner_list[i] = 'I-' + ner_list[i]

        tokens.append(token_list)
        ners.append(ner_list)
        spans.append(span_list)

    split_token_span_list = list(zip(split_text, span_text))
    span_ner_list = list(zip(spans, ners))

    sub_spans = []  # get sub spans from the full spans of the ner

    for each_span_ner_list in span_ner_list:
        # in full range ner e.g., [144, 150, 'GM-CSF', 'GP']
        count = 0
        # count is to keep track of the B, I, sub tags in the ner list
        for each_token in split_token_span_list:
            sub_spans_ = find_sub_span(each_token[1], each_span_ner_list[0])
            if sub_spans_:
                sub_spans.append([sub_spans_, each_span_ner_list[1][count]])
                count = count + 1

    for i, each_span_token in enumerate(split_token_span_list):
        for each_ner_span in sub_spans:
            if each_span_token[1] == each_ner_span[0]:
                arr[i] = ''.join(each_ner_span[1])

    return zip(split_text, arr)


In [4]:
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import csv


def find_sub_span(sub_span_range, full_spans_range):
    # if a sub span is present in full span return it
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range
    
    



In [5]:
result_path = '/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/en-pubmed-pmc-lg/public/'

from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer
from tqdm import tqdm
import csv
import pandas as pd

test_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/test.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])
dev_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/dev.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])
train_df = pd.read_csv('/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/train.csv', sep='\t', names = ['pmc_id','sentence', 'gt'])

In [6]:
gene_path= '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-BC2GM-en-pubmed-pmc-lg/best'
disease_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-NCBI-en-pubmed-pmc-lg/best'
organism_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/public/public-linnaeus-en-pubmed-pmc-lg/best'

GP_model = util.load_model_from_path(gene_path)
print('GP model loaded')

DS_model = util.load_model_from_path(disease_path)
print('DS model loaded')

OG_model = util.load_model_from_path(organism_path)
print('OG model loaded')

GP model loaded
DS model loaded
OG model loaded


In [28]:
####### Organisms ###################

train_result_file_name = 'spaCy_public_train_OG_IOB.csv'
dev_result_file_name = 'spaCy_public_dev_OG_IOB.csv'
test_result_file_name = 'spaCy_public_test_OG_IOB.csv'

with open(result_path + train_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(train_df.iterrows(), total=train_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = OG_model(text)

        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
with open(result_path + dev_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = OG_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
with open(result_path + test_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = OG_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')     



  0%|          | 0/80014 [00:00<?, ?it/s]

  0%|          | 29/80014 [00:00<04:36, 289.51it/s]

  0%|          | 59/80014 [00:00<04:34, 290.85it/s]

  0%|          | 100/80014 [00:00<04:11, 317.49it/s]

  0%|          | 141/80014 [00:00<03:55, 339.36it/s]

  0%|          | 181/80014 [00:00<03:45, 354.01it/s]

  0%|          | 213/80014 [00:00<03:55, 338.16it/s]

  0%|          | 247/80014 [00:00<03:57, 335.36it/s]

  0%|          | 279/80014 [00:00<04:18, 308.75it/s]

  0%|          | 309/80014 [00:00<04:25, 300.09it/s]

  0%|          | 339/80014 [00:01<04:34, 290.60it/s]

  0%|          | 368/80014 [00:01<04:35, 289.10it/s]

  0%|          | 398/80014 [00:01<04:34, 290.04it/s]

  1%|          | 427/80014 [00:01<04:41, 282.83it/s]

  1%|          | 456/80014 [00:01<04:41, 283.03it/s]

  1%|          | 485/80014 [00:01<04:57, 267.15it/s]

  1%|          | 512/80014 [00:01<04:56, 267.75it/s]

  1%|          | 540/80014 [00:01<04:55, 268.94it/s]

  1%|          | 571/80014 [00:01<04:46

  6%|▌         | 4574/80014 [00:14<03:51, 325.22it/s]

  6%|▌         | 4608/80014 [00:14<04:05, 307.15it/s]

  6%|▌         | 4640/80014 [00:14<04:13, 297.74it/s]

  6%|▌         | 4671/80014 [00:14<04:16, 293.52it/s]

  6%|▌         | 4701/80014 [00:14<04:20, 289.37it/s]

  6%|▌         | 4731/80014 [00:14<04:22, 286.55it/s]

  6%|▌         | 4760/80014 [00:14<04:23, 285.88it/s]

  6%|▌         | 4791/80014 [00:15<04:18, 291.49it/s]

  6%|▌         | 4831/80014 [00:15<03:57, 316.73it/s]

  6%|▌         | 4872/80014 [00:15<03:41, 339.40it/s]

  6%|▌         | 4913/80014 [00:15<03:31, 354.87it/s]

  6%|▌         | 4950/80014 [00:15<03:48, 327.86it/s]

  6%|▌         | 4984/80014 [00:15<04:00, 311.81it/s]

  6%|▋         | 5017/80014 [00:15<04:08, 302.34it/s]

  6%|▋         | 5048/80014 [00:15<04:24, 283.86it/s]

  6%|▋         | 5078/80014 [00:15<04:20, 287.42it/s]

  6%|▋         | 5108/80014 [00:16<04:19, 288.53it/s]

  6%|▋         | 5138/80014 [00:16<04:20, 286.98it/s]

  6%|▋    

 17%|█▋        | 13823/80014 [00:42<02:43, 404.72it/s]

 17%|█▋        | 13864/80014 [00:42<02:43, 405.46it/s]

 17%|█▋        | 13905/80014 [00:42<02:48, 393.02it/s]

 17%|█▋        | 13946/80014 [00:42<02:46, 396.04it/s]

 17%|█▋        | 13986/80014 [00:43<02:47, 394.78it/s]

 18%|█▊        | 14026/80014 [00:43<02:48, 392.14it/s]

 18%|█▊        | 14066/80014 [00:43<02:47, 393.19it/s]

 18%|█▊        | 14108/80014 [00:43<02:45, 399.18it/s]

 18%|█▊        | 14149/80014 [00:43<02:43, 401.97it/s]

 18%|█▊        | 14190/80014 [00:43<02:43, 403.50it/s]

 18%|█▊        | 14231/80014 [00:43<02:42, 404.50it/s]

 18%|█▊        | 14272/80014 [00:43<02:42, 405.43it/s]

 18%|█▊        | 14313/80014 [00:43<02:45, 396.82it/s]

 18%|█▊        | 14353/80014 [00:44<03:07, 349.92it/s]

 18%|█▊        | 14390/80014 [00:44<03:14, 336.89it/s]

 18%|█▊        | 14425/80014 [00:44<03:31, 309.89it/s]

 18%|█▊        | 14458/80014 [00:44<03:47, 288.48it/s]

 18%|█▊        | 14488/80014 [00:44<03:52, 281.8

 29%|██▊       | 22961/80014 [01:14<02:53, 329.61it/s]

 29%|██▊       | 22995/80014 [01:15<03:04, 309.84it/s]

 29%|██▉       | 23027/80014 [01:15<03:15, 291.98it/s]

 29%|██▉       | 23057/80014 [01:15<03:17, 288.67it/s]

 29%|██▉       | 23088/80014 [01:15<03:14, 293.33it/s]

 29%|██▉       | 23121/80014 [01:15<03:09, 300.39it/s]

 29%|██▉       | 23152/80014 [01:15<03:14, 292.22it/s]

 29%|██▉       | 23182/80014 [01:15<03:17, 287.61it/s]

 29%|██▉       | 23211/80014 [01:15<03:22, 279.85it/s]

 29%|██▉       | 23250/80014 [01:15<03:06, 304.26it/s]

 29%|██▉       | 23292/80014 [01:16<02:51, 331.52it/s]

 29%|██▉       | 23334/80014 [01:16<02:40, 353.00it/s]

 29%|██▉       | 23377/80014 [01:16<02:32, 372.31it/s]

 29%|██▉       | 23418/80014 [01:16<02:28, 381.68it/s]

 29%|██▉       | 23459/80014 [01:16<02:26, 387.35it/s]

 29%|██▉       | 23500/80014 [01:16<02:24, 391.58it/s]

 29%|██▉       | 23541/80014 [01:16<02:22, 396.00it/s]

 29%|██▉       | 23581/80014 [01:16<02:34, 365.1

 40%|████      | 32120/80014 [01:43<02:14, 355.20it/s]

 40%|████      | 32163/80014 [01:43<02:08, 373.29it/s]

 40%|████      | 32206/80014 [01:43<02:03, 387.17it/s]

 40%|████      | 32249/80014 [01:43<02:00, 397.80it/s]

 40%|████      | 32291/80014 [01:43<01:58, 402.84it/s]

 40%|████      | 32332/80014 [01:44<02:00, 395.24it/s]

 40%|████      | 32372/80014 [01:44<02:14, 353.35it/s]

 41%|████      | 32409/80014 [01:44<02:24, 328.92it/s]

 41%|████      | 32444/80014 [01:44<02:30, 315.22it/s]

 41%|████      | 32477/80014 [01:44<02:38, 300.74it/s]

 41%|████      | 32508/80014 [01:44<02:39, 297.98it/s]

 41%|████      | 32540/80014 [01:44<02:37, 301.77it/s]

 41%|████      | 32571/80014 [01:44<02:42, 292.08it/s]

 41%|████      | 32601/80014 [01:44<02:42, 292.51it/s]

 41%|████      | 32632/80014 [01:45<02:40, 294.78it/s]

 41%|████      | 32662/80014 [01:45<02:40, 294.74it/s]

 41%|████      | 32692/80014 [01:45<02:40, 294.25it/s]

 41%|████      | 32723/80014 [01:45<02:39, 296.4

...

 95%|█████████▌| 76062/80014 [04:06<00:09, 396.53it/s]

 95%|█████████▌| 76103/80014 [04:06<00:10, 372.00it/s]

 95%|█████████▌| 76144/80014 [04:06<00:10, 381.60it/s]

 95%|█████████▌| 76184/80014 [04:06<00:09, 385.14it/s]

 95%|█████████▌| 76226/80014 [04:06<00:09, 393.94it/s]

 95%|█████████▌| 76266/80014 [04:06<00:09, 394.68it/s]

 95%|█████████▌| 76306/80014 [04:06<00:09, 384.05it/s]

 95%|█████████▌| 76345/80014 [04:06<00:10, 354.62it/s]

 95%|█████████▌| 76382/80014 [04:07<00:10, 331.67it/s]

 96%|█████████▌| 76416/80014 [04:07<00:11, 318.04it/s]

 96%|█████████▌| 76449/80014 [04:07<00:11, 314.13it/s]

 96%|█████████▌| 76481/80014 [04:07<00:11, 301.05it/s]

 96%|█████████▌| 76512/80014 [04:07<00:11, 300.12it/s]

 96%|█████████▌| 76543/80014 [04:07<00:12, 288.98it/s]

 96%|█████████▌| 76581/80014 [04:07<00:11, 311.09it/s]

 96%|█████████▌| 76613/80014 [04:07<00:11, 300.02it/s]

 96%|█████████▌| 76644/80014 [04:07<00:11, 293.29it/s]

 96%|█████████▌| 76674/80014 [04:07<00:11, 287.4

 31%|███       | 4953/16108 [00:16<00:35, 312.15it/s]

 31%|███       | 4985/16108 [00:16<00:36, 307.77it/s]

 31%|███       | 5016/16108 [00:16<00:36, 305.55it/s]

 31%|███▏      | 5047/16108 [00:16<00:36, 305.72it/s]

 32%|███▏      | 5078/16108 [00:16<00:36, 303.37it/s]

 32%|███▏      | 5109/16108 [00:16<00:36, 301.70it/s]

 32%|███▏      | 5140/16108 [00:16<00:36, 297.77it/s]

 32%|███▏      | 5170/16108 [00:16<00:38, 285.58it/s]

 32%|███▏      | 5199/16108 [00:17<00:38, 284.20it/s]

 32%|███▏      | 5229/16108 [00:17<00:37, 287.84it/s]

 33%|███▎      | 5258/16108 [00:17<00:37, 288.14it/s]

 33%|███▎      | 5287/16108 [00:17<00:38, 283.18it/s]

 33%|███▎      | 5316/16108 [00:17<00:38, 283.52it/s]

 33%|███▎      | 5345/16108 [00:17<00:38, 279.28it/s]

 33%|███▎      | 5379/16108 [00:17<00:36, 293.31it/s]

 34%|███▎      | 5420/16108 [00:17<00:33, 319.63it/s]

 34%|███▍      | 5461/16108 [00:17<00:31, 341.93it/s]

 34%|███▍      | 5503/16108 [00:17<00:29, 359.82it/s]

 34%|███▍ 

 86%|████████▌ | 13818/16108 [00:44<00:07, 303.21it/s]

 86%|████████▌ | 13850/16108 [00:44<00:07, 306.31it/s]

 86%|████████▌ | 13882/16108 [00:44<00:07, 308.65it/s]

 86%|████████▋ | 13913/16108 [00:45<00:07, 292.11it/s]

 87%|████████▋ | 13943/16108 [00:45<00:07, 289.04it/s]

 87%|████████▋ | 13976/16108 [00:45<00:07, 297.81it/s]

 87%|████████▋ | 14012/16108 [00:45<00:06, 313.68it/s]

 87%|████████▋ | 14052/16108 [00:45<00:06, 333.96it/s]

 87%|████████▋ | 14090/16108 [00:45<00:05, 345.61it/s]

 88%|████████▊ | 14128/16108 [00:45<00:05, 352.92it/s]

 88%|████████▊ | 14164/16108 [00:45<00:05, 331.95it/s]

 88%|████████▊ | 14198/16108 [00:45<00:06, 311.90it/s]

 88%|████████▊ | 14230/16108 [00:46<00:06, 302.02it/s]

 89%|████████▊ | 14261/16108 [00:46<00:06, 304.01it/s]

 89%|████████▊ | 14292/16108 [00:46<00:05, 305.42it/s]

 89%|████████▉ | 14323/16108 [00:46<00:06, 285.89it/s]

 89%|████████▉ | 14353/16108 [00:46<00:06, 277.54it/s]

 89%|████████▉ | 14382/16108 [00:46<00:06, 270.4

 35%|███▍      | 6226/17957 [00:20<00:40, 289.71it/s]

 35%|███▍      | 6256/17957 [00:20<00:40, 288.69it/s]

 35%|███▌      | 6285/17957 [00:20<00:40, 286.68it/s]

 35%|███▌      | 6314/17957 [00:20<00:41, 279.80it/s]

 35%|███▌      | 6343/17957 [00:21<00:42, 275.51it/s]

 35%|███▌      | 6371/17957 [00:21<00:42, 272.05it/s]

 36%|███▌      | 6402/17957 [00:21<00:41, 281.46it/s]

 36%|███▌      | 6431/17957 [00:21<00:40, 282.03it/s]

 36%|███▌      | 6461/17957 [00:21<00:40, 286.46it/s]

 36%|███▌      | 6491/17957 [00:21<00:39, 289.84it/s]

 36%|███▋      | 6521/17957 [00:21<00:40, 279.30it/s]

 36%|███▋      | 6550/17957 [00:21<00:41, 272.92it/s]

 37%|███▋      | 6578/17957 [00:21<00:42, 268.08it/s]

 37%|███▋      | 6605/17957 [00:22<00:43, 259.40it/s]

 37%|███▋      | 6632/17957 [00:22<00:44, 254.36it/s]

 37%|███▋      | 6659/17957 [00:22<00:43, 258.72it/s]

 37%|███▋      | 6686/17957 [00:22<00:43, 261.88it/s]

 37%|███▋      | 6713/17957 [00:22<00:42, 261.90it/s]

 38%|███▊ 

 84%|████████▍ | 15043/17957 [00:49<00:10, 265.21it/s]

 84%|████████▍ | 15075/17957 [00:49<00:10, 278.35it/s]

 84%|████████▍ | 15118/17957 [00:49<00:09, 310.21it/s]

 84%|████████▍ | 15160/17957 [00:49<00:08, 335.23it/s]

 85%|████████▍ | 15203/17957 [00:49<00:07, 357.09it/s]

 85%|████████▍ | 15244/17957 [00:50<00:07, 370.74it/s]

 85%|████████▌ | 15283/17957 [00:50<00:08, 328.98it/s]

 85%|████████▌ | 15318/17957 [00:50<00:08, 306.96it/s]

 85%|████████▌ | 15351/17957 [00:50<00:08, 295.77it/s]

 86%|████████▌ | 15382/17957 [00:50<00:08, 290.84it/s]

 86%|████████▌ | 15412/17957 [00:50<00:09, 280.69it/s]

 86%|████████▌ | 15441/17957 [00:50<00:09, 274.75it/s]

 86%|████████▌ | 15469/17957 [00:50<00:09, 272.23it/s]

 86%|████████▋ | 15498/17957 [00:50<00:08, 275.79it/s]

 86%|████████▋ | 15527/17957 [00:51<00:08, 276.19it/s]

 87%|████████▋ | 15562/17957 [00:51<00:08, 293.54it/s]

 87%|████████▋ | 15596/17957 [00:51<00:07, 305.33it/s]

 87%|████████▋ | 15627/17957 [00:51<00:07, 305.3

In [29]:
####### Disease ###################

train_result_file_name = 'spaCy_public_train_DS_IOB.csv'
dev_result_file_name = 'spaCy_public_dev_DS_IOB.csv'
test_result_file_name = 'spaCy_public_test_DS_IOB.csv'

with open(result_path + train_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(train_df.iterrows(), total=train_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = DS_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
with open(result_path + dev_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = DS_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
with open(result_path + test_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = DS_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')        



  0%|          | 0/80014 [00:00<?, ?it/s]

  0%|          | 27/80014 [00:00<05:06, 261.31it/s]

  0%|          | 53/80014 [00:00<05:09, 258.66it/s]

  0%|          | 90/80014 [00:00<04:41, 283.67it/s]

  0%|          | 129/80014 [00:00<04:18, 308.60it/s]

  0%|          | 169/80014 [00:00<04:02, 329.45it/s]

  0%|          | 200/80014 [00:00<04:07, 322.25it/s]

  0%|          | 231/80014 [00:00<04:16, 311.42it/s]

  0%|          | 261/80014 [00:00<04:29, 295.58it/s]

  0%|          | 290/80014 [00:00<04:36, 287.92it/s]

  0%|          | 319/80014 [00:01<04:40, 284.29it/s]

  0%|          | 348/80014 [00:01<04:43, 280.98it/s]

  0%|          | 376/80014 [00:01<04:49, 275.48it/s]

  1%|          | 404/80014 [00:01<04:52, 271.79it/s]

  1%|          | 432/80014 [00:01<04:57, 267.67it/s]

  1%|          | 459/80014 [00:01<05:11, 255.26it/s]

  1%|          | 485/80014 [00:01<05:15, 252.02it/s]

  1%|          | 511/80014 [00:01<05:14, 252.74it/s]

  1%|          | 537/80014 [00:01<05:14,

 11%|█         | 8608/80014 [00:28<04:19, 275.48it/s]

 11%|█         | 8640/80014 [00:28<04:10, 285.47it/s]

 11%|█         | 8670/80014 [00:28<04:08, 287.38it/s]

 11%|█         | 8699/80014 [00:29<04:29, 264.42it/s]

 11%|█         | 8726/80014 [00:29<04:30, 263.58it/s]

 11%|█         | 8755/80014 [00:29<04:23, 270.07it/s]

 11%|█         | 8786/80014 [00:29<04:15, 278.92it/s]

 11%|█         | 8815/80014 [00:29<04:14, 279.69it/s]

 11%|█         | 8844/80014 [00:29<04:13, 281.17it/s]

 11%|█         | 8873/80014 [00:29<04:24, 269.06it/s]

 11%|█         | 8901/80014 [00:29<04:22, 270.66it/s]

 11%|█         | 8930/80014 [00:29<04:18, 275.23it/s]

 11%|█         | 8958/80014 [00:29<04:21, 272.03it/s]

 11%|█         | 8987/80014 [00:30<04:17, 275.41it/s]

 11%|█▏        | 9016/80014 [00:30<04:16, 277.09it/s]

 11%|█▏        | 9048/80014 [00:30<04:08, 286.07it/s]

 11%|█▏        | 9077/80014 [00:30<04:10, 283.49it/s]

 11%|█▏        | 9110/80014 [00:30<04:00, 294.37it/s]

 11%|█▏   

 22%|██▏       | 17843/80014 [00:57<02:53, 358.95it/s]

 22%|██▏       | 17886/80014 [00:57<02:45, 375.74it/s]

 22%|██▏       | 17925/80014 [00:57<02:51, 362.39it/s]

 22%|██▏       | 17965/80014 [00:57<02:46, 372.63it/s]

 23%|██▎       | 18006/80014 [00:57<02:42, 381.86it/s]

 23%|██▎       | 18045/80014 [00:57<02:50, 364.33it/s]

 23%|██▎       | 18082/80014 [00:57<02:56, 351.56it/s]

 23%|██▎       | 18118/80014 [00:57<03:04, 335.22it/s]

 23%|██▎       | 18152/80014 [00:57<03:09, 326.33it/s]

 23%|██▎       | 18185/80014 [00:58<03:10, 325.39it/s]

 23%|██▎       | 18218/80014 [00:58<03:15, 316.65it/s]

 23%|██▎       | 18250/80014 [00:58<03:20, 308.47it/s]

 23%|██▎       | 18282/80014 [00:58<03:27, 297.61it/s]

 23%|██▎       | 18312/80014 [00:58<03:30, 292.44it/s]

 23%|██▎       | 18342/80014 [00:58<03:30, 292.97it/s]

 23%|██▎       | 18372/80014 [00:58<03:40, 279.30it/s]

 23%|██▎       | 18403/80014 [00:58<03:34, 287.61it/s]

 23%|██▎       | 18440/80014 [00:58<03:20, 307.2

 34%|███▎      | 26974/80014 [01:25<02:16, 389.66it/s]

 34%|███▍      | 27014/80014 [01:25<02:25, 364.79it/s]

 34%|███▍      | 27052/80014 [01:25<02:34, 342.13it/s]

 34%|███▍      | 27087/80014 [01:26<02:48, 313.52it/s]

 34%|███▍      | 27120/80014 [01:26<02:57, 297.21it/s]

 34%|███▍      | 27155/80014 [01:26<02:50, 309.85it/s]

 34%|███▍      | 27192/80014 [01:26<02:43, 323.40it/s]

 34%|███▍      | 27225/80014 [01:26<02:48, 313.84it/s]

 34%|███▍      | 27257/80014 [01:26<02:48, 313.42it/s]

 34%|███▍      | 27289/80014 [01:26<02:51, 307.84it/s]

 34%|███▍      | 27321/80014 [01:26<02:54, 302.54it/s]

 34%|███▍      | 27352/80014 [01:26<02:56, 298.89it/s]

 34%|███▍      | 27383/80014 [01:27<02:56, 297.67it/s]

 34%|███▍      | 27414/80014 [01:27<02:55, 299.51it/s]

 34%|███▍      | 27447/80014 [01:27<02:51, 306.38it/s]

 34%|███▍      | 27478/80014 [01:27<02:51, 305.83it/s]

 34%|███▍      | 27511/80014 [01:27<02:49, 310.35it/s]

 34%|███▍      | 27551/80014 [01:27<02:37, 332.3

 45%|████▌     | 36155/80014 [01:54<03:01, 241.99it/s]

 45%|████▌     | 36183/80014 [01:54<02:54, 251.84it/s]

 45%|████▌     | 36216/80014 [01:54<02:41, 270.76it/s]

 45%|████▌     | 36246/80014 [01:54<02:37, 277.57it/s]

 45%|████▌     | 36275/80014 [01:54<02:36, 280.13it/s]

 45%|████▌     | 36308/80014 [01:54<02:29, 291.50it/s]

 45%|████▌     | 36348/80014 [01:55<02:18, 315.71it/s]

 45%|████▌     | 36389/80014 [01:55<02:08, 338.57it/s]

 46%|████▌     | 36424/80014 [01:55<02:14, 325.04it/s]

 46%|████▌     | 36458/80014 [01:55<02:18, 314.70it/s]

 46%|████▌     | 36500/80014 [01:55<02:08, 339.06it/s]

 46%|████▌     | 36542/80014 [01:55<02:01, 358.05it/s]

 46%|████▌     | 36584/80014 [01:55<01:56, 373.38it/s]

 46%|████▌     | 36623/80014 [01:55<01:57, 368.72it/s]

 46%|████▌     | 36661/80014 [01:55<02:04, 348.47it/s]

 46%|████▌     | 36697/80014 [01:56<02:12, 327.03it/s]

 46%|████▌     | 36731/80014 [01:56<02:17, 315.17it/s]

 46%|████▌     | 36765/80014 [01:56<02:14, 321.8

...

 89%|████████▉ | 71135/80014 [03:48<00:27, 319.97it/s]

 89%|████████▉ | 71171/80014 [03:48<00:26, 328.02it/s]

 89%|████████▉ | 71212/80014 [03:48<00:25, 348.02it/s]

 89%|████████▉ | 71248/80014 [03:48<00:25, 343.98it/s]

 89%|████████▉ | 71283/80014 [03:48<00:25, 343.77it/s]

 89%|████████▉ | 71325/80014 [03:48<00:23, 362.98it/s]

 89%|████████▉ | 71365/80014 [03:48<00:23, 373.28it/s]

 89%|████████▉ | 71407/80014 [03:48<00:22, 383.80it/s]

 89%|████████▉ | 71448/80014 [03:49<00:21, 390.14it/s]

 89%|████████▉ | 71489/80014 [03:49<00:21, 393.72it/s]

 89%|████████▉ | 71529/80014 [03:49<00:21, 388.00it/s]

 89%|████████▉ | 71568/80014 [03:49<00:23, 366.36it/s]

 89%|████████▉ | 71605/80014 [03:49<00:23, 354.30it/s]

 90%|████████▉ | 71641/80014 [03:49<00:25, 325.08it/s]

 90%|████████▉ | 71675/80014 [03:49<00:26, 314.11it/s]

 90%|████████▉ | 71707/80014 [03:49<00:27, 306.64it/s]

 90%|████████▉ | 71739/80014 [03:49<00:28, 295.36it/s]

 90%|████████▉ | 71769/80014 [03:50<00:28, 288.8

  0%|          | 31/16108 [00:00<00:52, 308.27it/s]

  0%|          | 60/16108 [00:00<00:53, 300.66it/s]

  1%|          | 91/16108 [00:00<00:53, 301.41it/s]

  1%|          | 123/16108 [00:00<00:52, 304.45it/s]

  1%|          | 153/16108 [00:00<00:53, 300.97it/s]

  1%|          | 182/16108 [00:00<00:53, 296.73it/s]

  1%|▏         | 216/16108 [00:00<00:51, 306.83it/s]

  2%|▏         | 249/16108 [00:00<00:50, 313.22it/s]

  2%|▏         | 279/16108 [00:00<00:52, 302.66it/s]

  2%|▏         | 310/16108 [00:01<00:51, 304.79it/s]

  2%|▏         | 340/16108 [00:01<00:52, 302.46it/s]

  2%|▏         | 370/16108 [00:01<00:52, 300.22it/s]

  2%|▏         | 402/16108 [00:01<00:51, 305.72it/s]

  3%|▎         | 438/16108 [00:01<00:49, 319.46it/s]

  3%|▎         | 476/16108 [00:01<00:46, 333.42it/s]

  3%|▎         | 517/16108 [00:01<00:44, 349.91it/s]

  3%|▎         | 553/16108 [00:01<00:45, 344.78it/s]

  4%|▎         | 588/16108 [00:01<00:44, 346.05it/s]

  4%|▍         | 623/16108 [00:

 54%|█████▍    | 8683/16108 [00:28<00:26, 281.12it/s]

 54%|█████▍    | 8713/16108 [00:29<00:25, 284.78it/s]

 54%|█████▍    | 8744/16108 [00:29<00:25, 289.19it/s]

 54%|█████▍    | 8773/16108 [00:29<00:25, 289.28it/s]

 55%|█████▍    | 8813/16108 [00:29<00:23, 313.79it/s]

 55%|█████▍    | 8850/16108 [00:29<00:22, 327.22it/s]

 55%|█████▌    | 8884/16108 [00:29<00:22, 314.70it/s]

 55%|█████▌    | 8916/16108 [00:29<00:23, 306.45it/s]

 56%|█████▌    | 8948/16108 [00:29<00:24, 297.33it/s]

 56%|█████▌    | 8979/16108 [00:29<00:24, 288.13it/s]

 56%|█████▌    | 9009/16108 [00:30<00:24, 287.97it/s]

 56%|█████▌    | 9039/16108 [00:30<00:25, 279.67it/s]

 56%|█████▋    | 9080/16108 [00:30<00:22, 308.53it/s]

 57%|█████▋    | 9121/16108 [00:30<00:21, 332.57it/s]

 57%|█████▋    | 9161/16108 [00:30<00:19, 349.24it/s]

 57%|█████▋    | 9202/16108 [00:30<00:18, 364.83it/s]

 57%|█████▋    | 9241/16108 [00:30<00:18, 371.90it/s]

 58%|█████▊    | 9282/16108 [00:30<00:17, 381.72it/s]

 58%|█████

  7%|▋         | 1291/17957 [00:04<00:59, 281.91it/s]

  7%|▋         | 1320/17957 [00:04<00:59, 278.27it/s]

  8%|▊         | 1351/17957 [00:04<00:58, 285.97it/s]

  8%|▊         | 1384/17957 [00:04<00:55, 297.81it/s]

  8%|▊         | 1414/17957 [00:04<00:56, 295.08it/s]

  8%|▊         | 1444/17957 [00:04<01:02, 264.05it/s]

  8%|▊         | 1473/17957 [00:05<01:00, 270.60it/s]

  8%|▊         | 1501/17957 [00:05<01:01, 268.52it/s]

  9%|▊         | 1529/17957 [00:05<01:01, 265.92it/s]

  9%|▊         | 1559/17957 [00:05<00:59, 274.81it/s]

  9%|▉         | 1589/17957 [00:05<00:58, 277.52it/s]

  9%|▉         | 1617/17957 [00:05<01:02, 262.70it/s]

  9%|▉         | 1645/17957 [00:05<01:01, 267.13it/s]

  9%|▉         | 1673/17957 [00:05<01:00, 268.83it/s]

  9%|▉         | 1702/17957 [00:05<00:59, 274.24it/s]

 10%|▉         | 1739/17957 [00:06<00:54, 295.99it/s]

 10%|▉         | 1779/17957 [00:06<00:50, 320.04it/s]

 10%|█         | 1819/17957 [00:06<00:47, 339.52it/s]

 10%|█    

 57%|█████▋    | 10317/17957 [00:33<00:19, 398.55it/s]

 58%|█████▊    | 10358/17957 [00:33<00:19, 396.79it/s]

 58%|█████▊    | 10398/17957 [00:33<00:20, 367.46it/s]

 58%|█████▊    | 10436/17957 [00:33<00:22, 331.49it/s]

 58%|█████▊    | 10471/17957 [00:33<00:26, 285.37it/s]

 58%|█████▊    | 10502/17957 [00:33<00:27, 272.51it/s]

 59%|█████▊    | 10531/17957 [00:33<00:27, 272.67it/s]

 59%|█████▉    | 10563/17957 [00:34<00:26, 283.84it/s]

 59%|█████▉    | 10593/17957 [00:34<00:26, 281.53it/s]

 59%|█████▉    | 10622/17957 [00:34<00:26, 276.39it/s]

 59%|█████▉    | 10652/17957 [00:34<00:25, 283.01it/s]

 59%|█████▉    | 10681/17957 [00:34<00:26, 279.29it/s]

 60%|█████▉    | 10710/17957 [00:34<00:25, 281.44it/s]

 60%|█████▉    | 10739/17957 [00:34<00:25, 281.39it/s]

 60%|█████▉    | 10771/17957 [00:34<00:24, 290.51it/s]

 60%|██████    | 10801/17957 [00:34<00:24, 289.29it/s]

 60%|██████    | 10832/17957 [00:34<00:24, 292.68it/s]

 60%|██████    | 10862/17957 [00:35<00:24, 292.7

In [30]:
####### Genes ###################

train_result_file_name = 'spaCy_public_train_GP_IOB.csv'
dev_result_file_name = 'spaCy_public_dev_GP_IOB.csv'
test_result_file_name = 'spaCy_public_test_GP_IOB.csv'

with open(result_path + train_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(train_df.iterrows(), total=train_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = GP_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')   

        
        
with open(result_path + dev_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = GP_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
with open(result_path + test_result_file_name, 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

    for index_, each_annotation in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        text = each_annotation['sentence']
        ml_ner = []
        sentence = GP_model(text)
        
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)
            
        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')



  0%|          | 0/80014 [00:00<?, ?it/s]

  0%|          | 27/80014 [00:00<05:03, 263.78it/s]

  0%|          | 54/80014 [00:00<05:04, 262.92it/s]

  0%|          | 92/80014 [00:00<04:36, 288.79it/s]

  0%|          | 131/80014 [00:00<04:16, 311.47it/s]

  0%|          | 170/80014 [00:00<04:01, 329.97it/s]

  0%|          | 201/80014 [00:00<04:06, 323.21it/s]

  0%|          | 231/80014 [00:00<04:12, 315.49it/s]

  0%|          | 261/80014 [00:00<04:28, 297.01it/s]

  0%|          | 290/80014 [00:00<04:35, 289.71it/s]

  0%|          | 319/80014 [00:01<04:38, 285.93it/s]

  0%|          | 348/80014 [00:01<04:41, 283.07it/s]

  0%|          | 377/80014 [00:01<04:45, 279.22it/s]

  1%|          | 405/80014 [00:01<04:49, 274.56it/s]

  1%|          | 433/80014 [00:01<04:54, 270.11it/s]

  1%|          | 460/80014 [00:01<05:21, 247.65it/s]

  1%|          | 486/80014 [00:01<05:23, 245.64it/s]

  1%|          | 512/80014 [00:01<05:20, 247.80it/s]

  1%|          | 538/80014 [00:01<05:18,

 11%|█         | 8704/80014 [00:28<04:19, 274.57it/s]

 11%|█         | 8732/80014 [00:28<04:19, 274.60it/s]

 11%|█         | 8762/80014 [00:28<04:15, 278.93it/s]

 11%|█         | 8792/80014 [00:29<04:11, 282.79it/s]

 11%|█         | 8821/80014 [00:29<04:13, 280.99it/s]

 11%|█         | 8850/80014 [00:29<04:14, 279.59it/s]

 11%|█         | 8879/80014 [00:29<04:13, 280.87it/s]

 11%|█         | 8908/80014 [00:29<04:12, 281.15it/s]

 11%|█         | 8937/80014 [00:29<04:16, 277.39it/s]

 11%|█         | 8965/80014 [00:29<04:23, 269.40it/s]

 11%|█         | 8995/80014 [00:29<04:16, 276.74it/s]

 11%|█▏        | 9023/80014 [00:29<04:16, 276.61it/s]

 11%|█▏        | 9053/80014 [00:29<04:11, 282.65it/s]

 11%|█▏        | 9082/80014 [00:30<04:09, 284.68it/s]

 11%|█▏        | 9111/80014 [00:30<04:16, 275.96it/s]

 11%|█▏        | 9141/80014 [00:30<04:11, 281.75it/s]

 11%|█▏        | 9171/80014 [00:30<04:08, 285.28it/s]

 11%|█▏        | 9201/80014 [00:30<04:06, 287.49it/s]

 12%|█▏   

 22%|██▏       | 17972/80014 [00:57<02:45, 375.00it/s]

 23%|██▎       | 18012/80014 [00:57<02:43, 378.51it/s]

 23%|██▎       | 18051/80014 [00:57<02:50, 363.80it/s]

 23%|██▎       | 18091/80014 [00:57<02:46, 371.63it/s]

 23%|██▎       | 18129/80014 [00:57<02:56, 349.98it/s]

 23%|██▎       | 18165/80014 [00:57<03:02, 338.98it/s]

 23%|██▎       | 18200/80014 [00:57<03:08, 328.60it/s]

 23%|██▎       | 18234/80014 [00:57<03:29, 295.07it/s]

 23%|██▎       | 18265/80014 [00:58<03:31, 292.21it/s]

 23%|██▎       | 18295/80014 [00:58<03:37, 284.13it/s]

 23%|██▎       | 18324/80014 [00:58<03:38, 281.77it/s]

 23%|██▎       | 18355/80014 [00:58<03:34, 287.17it/s]

 23%|██▎       | 18386/80014 [00:58<03:31, 291.82it/s]

 23%|██▎       | 18418/80014 [00:58<03:27, 297.55it/s]

 23%|██▎       | 18460/80014 [00:58<03:09, 325.11it/s]

 23%|██▎       | 18502/80014 [00:58<02:56, 348.18it/s]

 23%|██▎       | 18541/80014 [00:58<02:51, 358.13it/s]

 23%|██▎       | 18578/80014 [00:58<03:04, 332.3

 34%|███▍      | 27049/80014 [01:25<02:38, 334.33it/s]

 34%|███▍      | 27084/80014 [01:25<02:55, 302.42it/s]

 34%|███▍      | 27116/80014 [01:26<03:05, 284.67it/s]

 34%|███▍      | 27150/80014 [01:26<02:57, 297.46it/s]

 34%|███▍      | 27187/80014 [01:26<02:47, 315.79it/s]

 34%|███▍      | 27220/80014 [01:26<02:54, 301.91it/s]

 34%|███▍      | 27251/80014 [01:26<02:54, 303.11it/s]

 34%|███▍      | 27282/80014 [01:26<02:56, 299.17it/s]

 34%|███▍      | 27313/80014 [01:26<02:57, 296.59it/s]

 34%|███▍      | 27343/80014 [01:26<02:58, 294.47it/s]

 34%|███▍      | 27373/80014 [01:26<03:00, 291.09it/s]

 34%|███▍      | 27403/80014 [01:26<03:00, 291.88it/s]

 34%|███▍      | 27435/80014 [01:27<02:55, 298.93it/s]

 34%|███▍      | 27466/80014 [01:27<02:55, 298.86it/s]

 34%|███▍      | 27498/80014 [01:27<02:52, 303.64it/s]

 34%|███▍      | 27531/80014 [01:27<02:49, 309.02it/s]

 34%|███▍      | 27566/80014 [01:27<02:44, 318.33it/s]

 34%|███▍      | 27598/80014 [01:27<02:46, 315.2

 45%|████▌     | 36102/80014 [01:54<03:42, 197.48it/s]

 45%|████▌     | 36128/80014 [01:54<03:26, 212.74it/s]

 45%|████▌     | 36156/80014 [01:54<03:12, 227.45it/s]

 45%|████▌     | 36184/80014 [01:54<03:02, 240.48it/s]

 45%|████▌     | 36217/80014 [01:54<02:47, 261.00it/s]

 45%|████▌     | 36247/80014 [01:55<02:42, 270.16it/s]

 45%|████▌     | 36276/80014 [01:55<02:39, 274.29it/s]

 45%|████▌     | 36310/80014 [01:55<02:30, 290.72it/s]

 45%|████▌     | 36352/80014 [01:55<02:16, 319.97it/s]

 45%|████▌     | 36395/80014 [01:55<02:06, 344.96it/s]

 46%|████▌     | 36432/80014 [01:55<02:15, 322.57it/s]

 46%|████▌     | 36466/80014 [01:55<02:15, 320.34it/s]

 46%|████▌     | 36507/80014 [01:55<02:07, 342.17it/s]

 46%|████▌     | 36549/80014 [01:55<02:00, 359.96it/s]

 46%|████▌     | 36591/80014 [01:56<01:56, 374.12it/s]

 46%|████▌     | 36630/80014 [01:56<02:00, 358.59it/s]

 46%|████▌     | 36667/80014 [01:56<02:06, 341.55it/s]

 46%|████▌     | 36702/80014 [01:56<02:13, 323.8

...

 89%|████████▊ | 70842/80014 [03:48<00:30, 305.19it/s]

 89%|████████▊ | 70874/80014 [03:49<00:31, 287.80it/s]

 89%|████████▊ | 70904/80014 [03:49<00:33, 274.43it/s]

 89%|████████▊ | 70934/80014 [03:49<00:32, 281.53it/s]

 89%|████████▊ | 70963/80014 [03:49<00:31, 283.04it/s]

 89%|████████▊ | 70992/80014 [03:49<00:31, 282.10it/s]

 89%|████████▉ | 71021/80014 [03:49<00:31, 283.76it/s]

 89%|████████▉ | 71053/80014 [03:49<00:30, 291.94it/s]

 89%|████████▉ | 71090/80014 [03:49<00:28, 308.63it/s]

 89%|████████▉ | 71122/80014 [03:49<00:28, 308.84it/s]

 89%|████████▉ | 71162/80014 [03:49<00:26, 330.18it/s]

 89%|████████▉ | 71199/80014 [03:50<00:26, 338.99it/s]

 89%|████████▉ | 71240/80014 [03:50<00:24, 356.07it/s]

 89%|████████▉ | 71277/80014 [03:50<00:24, 352.48it/s]

 89%|████████▉ | 71318/80014 [03:50<00:23, 366.37it/s]

 89%|████████▉ | 71357/80014 [03:50<00:23, 371.10it/s]

 89%|████████▉ | 71396/80014 [03:50<00:22, 375.33it/s]

 89%|████████▉ | 71436/80014 [03:50<00:22, 380.0

100%|█████████▉| 79622/80014 [04:17<00:01, 351.89it/s]

100%|█████████▉| 79658/80014 [04:17<00:01, 328.42it/s]

100%|█████████▉| 79692/80014 [04:17<00:00, 322.02it/s]

100%|█████████▉| 79725/80014 [04:17<00:00, 319.08it/s]

100%|█████████▉| 79758/80014 [04:17<00:00, 299.89it/s]

100%|█████████▉| 79789/80014 [04:18<00:00, 292.64it/s]

100%|█████████▉| 79819/80014 [04:18<00:00, 285.85it/s]

100%|█████████▉| 79848/80014 [04:18<00:00, 278.34it/s]

100%|█████████▉| 79877/80014 [04:18<00:00, 279.48it/s]

100%|█████████▉| 79906/80014 [04:18<00:00, 280.64it/s]

100%|█████████▉| 79937/80014 [04:18<00:00, 287.19it/s]

100%|█████████▉| 79966/80014 [04:18<00:00, 273.82it/s]

100%|██████████| 80014/80014 [04:18<00:00, 309.12it/s]


  0%|          | 0/16108 [00:00<?, ?it/s]

  0%|          | 32/16108 [00:00<00:51, 310.07it/s]

  0%|          | 61/16108 [00:00<00:53, 302.56it/s]

  1%|          | 93/16108 [00:00<00:52, 305.97it/s]

  1%|          | 124/16108 [00:00<00:52, 307.12it/s]

  1%|          

 50%|█████     | 8130/16108 [00:27<00:28, 278.49it/s]

 51%|█████     | 8159/16108 [00:27<00:30, 259.84it/s]

 51%|█████     | 8188/16108 [00:27<00:29, 265.95it/s]

 51%|█████     | 8217/16108 [00:27<00:29, 270.90it/s]

 51%|█████     | 8246/16108 [00:28<00:28, 276.24it/s]

 51%|█████▏    | 8276/16108 [00:28<00:27, 282.24it/s]

 52%|█████▏    | 8306/16108 [00:28<00:27, 286.82it/s]

 52%|█████▏    | 8336/16108 [00:28<00:27, 287.34it/s]

 52%|█████▏    | 8365/16108 [00:28<00:26, 287.43it/s]

 52%|█████▏    | 8396/16108 [00:28<00:26, 293.76it/s]

 52%|█████▏    | 8429/16108 [00:28<00:25, 302.52it/s]

 53%|█████▎    | 8460/16108 [00:28<00:26, 291.41it/s]

 53%|█████▎    | 8490/16108 [00:28<00:26, 286.38it/s]

 53%|█████▎    | 8519/16108 [00:29<00:26, 284.65it/s]

 53%|█████▎    | 8548/16108 [00:29<00:26, 285.31it/s]

 53%|█████▎    | 8579/16108 [00:29<00:25, 291.83it/s]

 53%|█████▎    | 8609/16108 [00:29<00:25, 288.99it/s]

 54%|█████▎    | 8638/16108 [00:29<00:26, 284.89it/s]

 54%|█████

  4%|▍         | 700/17957 [00:02<00:52, 330.72it/s]

  4%|▍         | 734/17957 [00:02<00:53, 321.10it/s]

  4%|▍         | 767/17957 [00:02<00:54, 313.69it/s]

  4%|▍         | 799/17957 [00:02<00:56, 302.17it/s]

  5%|▍         | 830/17957 [00:02<00:56, 304.17it/s]

  5%|▍         | 862/17957 [00:02<00:55, 306.37it/s]

  5%|▍         | 893/17957 [00:03<00:56, 302.69it/s]

  5%|▌         | 926/17957 [00:03<00:55, 307.95it/s]

  5%|▌         | 957/17957 [00:03<00:57, 296.70it/s]

  5%|▌         | 987/17957 [00:03<00:58, 290.70it/s]

  6%|▌         | 1019/17957 [00:03<00:57, 294.91it/s]

  6%|▌         | 1049/17957 [00:03<00:57, 292.67it/s]

  6%|▌         | 1079/17957 [00:03<00:59, 285.66it/s]

  6%|▌         | 1108/17957 [00:03<01:00, 280.61it/s]

  6%|▋         | 1137/17957 [00:03<00:59, 281.61it/s]

  6%|▋         | 1167/17957 [00:03<00:58, 285.91it/s]

  7%|▋         | 1196/17957 [00:04<00:58, 285.00it/s]

  7%|▋         | 1227/17957 [00:04<00:57, 291.20it/s]

  7%|▋         | 125

 53%|█████▎    | 9509/17957 [00:31<00:29, 289.07it/s]

 53%|█████▎    | 9551/17957 [00:31<00:26, 317.64it/s]

 53%|█████▎    | 9592/17957 [00:31<00:24, 340.48it/s]

 54%|█████▎    | 9632/17957 [00:31<00:24, 340.47it/s]

 54%|█████▍    | 9674/17957 [00:31<00:23, 359.78it/s]

 54%|█████▍    | 9716/17957 [00:31<00:21, 375.00it/s]

 54%|█████▍    | 9758/17957 [00:31<00:21, 385.99it/s]

 55%|█████▍    | 9799/17957 [00:32<00:20, 392.80it/s]

 55%|█████▍    | 9839/17957 [00:32<00:22, 363.81it/s]

 55%|█████▌    | 9877/17957 [00:32<00:23, 342.18it/s]

 55%|█████▌    | 9913/17957 [00:32<00:25, 310.61it/s]

 55%|█████▌    | 9946/17957 [00:32<00:26, 302.02it/s]

 56%|█████▌    | 9978/17957 [00:32<00:27, 293.22it/s]

 56%|█████▌    | 10009/17957 [00:32<00:26, 296.93it/s]

 56%|█████▌    | 10051/17957 [00:32<00:24, 325.47it/s]

 56%|█████▌    | 10093/17957 [00:33<00:22, 346.98it/s]

 56%|█████▋    | 10135/17957 [00:33<00:21, 364.00it/s]

 57%|█████▋    | 10173/17957 [00:33<00:21, 354.31it/s]

 57%|